# Originality Algo

## Algorithm to Automate Originality Scoring

### Import Packages

In [1]:
import nltk
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
import re
from functools import reduce
import openpyxl
import xlsxwriter

from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.util import ngrams
from nltk import ngrams, FreqDist
from nltk.lm import NgramCounter
import string
import gensim
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors

from spacy.lang.en.stop_words import STOP_WORDS

from collections import Counter
import itertools

from sklearn.metrics.pairwise import cosine_similarity

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer

### Put Data from Excel Sheet into Dataframes

In [2]:
# individual df's for each sheet

# when on pc
data_cup = pd.read_excel("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/autdata_test.xlsx", sheet_name = "Cup")
data_key = pd.read_excel("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/autdata_test.xlsx", sheet_name = "Key")
data_rope = pd.read_excel("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/autdata_test.xlsx", sheet_name = "Rope")
data_brick = pd.read_excel("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/autdata_test.xlsx", sheet_name = "Brick")
data_chair = pd.read_excel("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/autdata_test.xlsx", sheet_name = "Chair")
data_pencil = pd.read_excel("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/autdata_test.xlsx", sheet_name = "Pencil")
data_shoe = pd.read_excel("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/autdata_test.xlsx", sheet_name = "Shoe")

# when on mac
# data_cup = pd.read_excel("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/autdata_test.xlsx", sheet_name = "Cup", engine='openpyxl')
# data_key = pd.read_excel("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/autdata_test.xlsx", sheet_name = "Key", engine='openpyxl')
# data_rope = pd.read_excel("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/autdata_test.xlsx", sheet_name = "Rope", engine='openpyxl')
# data_brick = pd.read_excel("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/autdata_test.xlsx", sheet_name = "Brick", engine='openpyxl')
# data_chair = pd.read_excel("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/autdata_test.xlsx", sheet_name = "Chair", engine='openpyxl')
# data_pencil = pd.read_excel("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/autdata_test.xlsx", sheet_name = "Pencil", engine='openpyxl')
# data_shoe = pd.read_excel("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/autdata_test.xlsx", sheet_name = "Shoe", engine='openpyxl')

### Preprocessing

In [3]:
# nltk corpus stop words
stopwords_nltk = stopwords.words('english')
# spacy stop words
stopwords_spacy = STOP_WORDS

In [4]:
# method to clean the responses
def process_text(text, stopwords_list, remove_sw, join_list, stem = True):
    # tokenize text, lemmanize words, removing punctuation, remove stop words, lowercase all words

    # hardcorded for special situations
    text = re.sub("wedging","wedge", text)
    text = re.sub("exersizing","exercising", text)
    
    text = re.sub("/|-"," ", text)
    text = text.translate(str.maketrans('','',string.punctuation))
    tokens = word_tokenize(text)

    if remove_sw:
        tokens = [word for word in tokens if word not in stopwords_list]

    tokens = [w.lower() for w in tokens]

    if stem:
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(t) for t in tokens]
#         stemmer = PorterStemmer()
#         tokens = [stemmer.stem(t) for t in tokens]

    if join_list:
        tokens = ' '.join(tokens)
 
    return tokens

### General Functions

In [5]:
# method to get a list of participants
def get_id_list(df):
    id_list = df['id'].unique()
    id_list = sorted(id_list)
    return id_list

In [6]:
# method to add a new column
# new column are cleaned responses
def get_cleaned_responses(df, stopwords_list, remove_sw, join_list):
    # id_df = df[df.id == id]
    df_processed = df.copy(deep=True)
    responses = df['response'].tolist()

    # make list of processed responses
    for response in range(len(responses)):
        responses[response] = process_text(responses[response], stopwords_list, remove_sw, join_list, True)

    # add list as column in df
    df_processed['response_processed'] = responses

    return df_processed

### Word2Vec Models for Embeddings

In [7]:
# load pretrained model
word_model_twitter25 = api.load("glove-twitter-25")

# on pc
# word_model_google = KeyedVectors.load_word2vec_format("C:/Users/jhec8/Documents/Northwestern_SROP/GoogleNews-vectors-negative300.bin", binary=True)

# on mac
# word_model = KeyedVectors.load_word2vec_format("/Users/johnhenrycruz/Desktop/Northwestern_SROP/GoogleNews-vectors-negative300.bin", binary=True)

In [8]:
# create dictionary of counts for each word in model
twitter25_dict = {}
for i in range(len(word_model_twitter25)):
    twitter25_dict[word_model_twitter25.index_to_key[i]] = word_model_twitter25.key_to_index[word_model_twitter25.index_to_key[i]]

In [9]:
# get the frequency of each word in dictionary
total_words = 0
for key in twitter25_dict:
    total_words = total_words + twitter25_dict[key]
    
for key in twitter25_dict:
    twitter25_dict[key] = twitter25_dict[key]/total_words

## Originality Algo 1
### Term Frequency Only

In [171]:
def get_tf_dict(responses):
    cv = CountVectorizer()   
    cv_fit = cv.fit_transform(responses)    
    word_list = cv.get_feature_names()

    # [0] here to get a 1d-array for iteration by the zip function 
    count_list = np.asarray(cv_fit.sum(axis=0))[0]

    tf_dict = dict(zip(word_list, count_list))
    
    total_words = 0
    for key in tf_dict:
        total_words = total_words + tf_dict[key]
        
    for key in tf_dict:
        tf_dict[key] = np.log(1 + tf_dict[key]/total_words)
        
    tf_dict = dict(sorted(tf_dict.items(), key=lambda item: item[1], reverse=True))
                
    return tf_dict

In [176]:
def get_tf_sum(tf_dict, response):
    tf_sum = 0
    for term in range(len(response)):
        tf_sum = tf_sum + tf_dict[response[term]]
        
    if tf_sum == 0:
        tf_sum = np.nan
        
    return tf_sum

In [177]:
def get_tf_sum_list(tf_dict, responses):
    tf_sum_list = []
    
    for response in responses:
        tf_sum_list.append(get_tf_sum(tf_dict, response))
            
    return tf_sum_list

In [178]:
def get_originality_tf_only(df, stopwords_list, remove_sw, join_list):
    originality_rating_df = get_cleaned_responses(df, stopwords_list, remove_sw, join_list)
    responses = originality_rating_df['response_processed'].tolist()
    responses_tokenized = [item for sublist in responses for item in sublist]

    tf_dict = get_tf_dict(responses_tokenized)
    
    tf_sum_list = get_tf_sum_list(tf_dict, responses)

    originality_rating_df['tf_sum'] = tf_sum_list
    
    print(tf_dict)
    
    return originality_rating_df

In [179]:
get_originality_tf_only(data_brick, stopwords_spacy, True, False)

{'build': 0.0905140075408319, 'weapon': 0.061243625240718594, 'house': 0.05129329438755048, 'weight': 0.05129329438755048, 'break': 0.041242958534049, 'paperweight': 0.041242958534049, 'use': 0.041242958534049, 'could': 0.03109058707003118, 'thing': 0.03109058707003118, 'throw': 0.03109058707003118, 'door': 0.020834086902842053, 'line': 0.020834086902842053, 'road': 0.020834086902842053, 'window': 0.020834086902842053, 'anchor': 0.010471299867295437, 'anvil': 0.010471299867295437, 'art': 0.010471299867295437, 'boat': 0.010471299867295437, 'bowl': 0.010471299867295437, 'canvas': 0.010471299867295437, 'carve': 0.010471299867295437, 'combination': 0.010471299867295437, 'cooking': 0.010471299867295437, 'demolish': 0.010471299867295437, 'fire': 0.010471299867295437, 'fireplace': 0.010471299867295437, 'foundation': 0.010471299867295437, 'hard': 0.010471299867295437, 'hit': 0.010471299867295437, 'hold': 0.010471299867295437, 'holding': 0.010471299867295437, 'karate': 0.010471299867295437, 'li

,id,stim,response,response_processed,tf_sum
0,1476,Brick,as a foundation,[foundation],0.010471
1,1476,Brick,as a weapon,[weapon],0.061244
2,1476,Brick,as a bowl,[bowl],0.010471
3,1718,Brick,build a house,"[build, house]",0.141807
4,1718,Brick,break a window,"[break, window]",0.062077
5,1718,Brick,line a fireplace,"[line, fireplace]",0.031305
6,1718,Brick,line a road,"[line, road]",0.041668
7,1691,Brick,build house,"[build, house]",0.141807
8,1691,Brick,break window,"[break, window]",0.062077
9,1691,Brick,weight for workout,"[weight, workout]",0.061765


## Originality Algo 2
### tf-idf scikit-learn
### participant as document

In [180]:
def tfidf_scikit_learn(cleaned_responses):
    tfIdfVectorizer=TfidfVectorizer(use_idf=True)
    tfIdf = tfIdfVectorizer.fit_transform(cleaned_responses)
    df = pd.DataFrame(tfIdf.toarray(), columns=tfIdfVectorizer.get_feature_names())
#     df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
#     df = df.sort_values('TF-IDF', ascending=False)
    display(df)
    
#     feature_names = tfIdfVectorizer.get_feature_names()
#     for col in tfIdf.nonzero()[1]:
#         print (feature_names[col], ' - ', tfIdf[0, col])

    print(tfIdf)

In [181]:
def get_originality_tfidf_scikit_learn(df, stopwords_list, remove_sw, join_list):
    originality_rating_df = get_cleaned_responses(df, stopwords_list, remove_sw, join_list)
    responses = []
    
    id_list = get_id_list(df)
    
    for participant in id_list:
        temp_df = originality_rating_df.loc[originality_rating_df['id'] == participant]
        temp_list = temp_df['response_processed'].tolist()
        temp_list = ' '.join(temp_list)
        responses.append(temp_list)
        
    print(responses)
    
    tfidf_scikit_learn(responses)   

In [184]:
get_originality_tfidf_scikit_learn(data_brick, stopwords_spacy, True, True)

['foundation weapon bowl', 'house fire pit weapon break sidewalk paint', 'build hit people paperweight', 'build house break window weight workout weight holding wind karate', 'build house use paperweight break prop martial art use step stool', 'build house break window line fireplace line road', 'throw thing smash thing paper weight weight lifting', 'door stop weapon canvas', 'build use weapon throw', 'build throw demolish paperweight', 'build use weapon paperweight anchor small boat combination rope obviously', 'build house hold open door road carve name cooking', 'used build thing could thrown weapon could weight could hard surface like anvil']


,anchor,anvil,art,boat,bowl,break,build,canvas,carve,combination,...,thing,throw,thrown,use,used,weapon,weight,wind,window,workout
0,0.000000,0.000000,0.000000,0.000000,0.655075,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.376501,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.302181,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.252086,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.277098,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.241237,0.158851,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.535518,0.350145,0.301952,0.350145
4,0.000000,0.000000,0.335437,0.000000,0.000000,0.231103,0.152178,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.513023,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.250556,0.164987,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.313617,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.578176,0.256351,0.000000,0.000000,0.000000,0.000000,0.512703,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.570358,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.327811,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.347367,0.000000,0.000000,0.000000,...,0.000000,0.585523,0.000000,0.585523,0.000000,0.440071,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.301426,0.000000,0.000000,0.000000,...,0.000000,0.508085,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


  (0, 4)	0.655075136305677
  (0, 50)	0.376501170766038
  (0, 16)	0.655075136305677
  (1, 30)	0.43860410751211154
  (1, 38)	0.43860410751211154
  (1, 5)	0.3021814542943412
  (1, 34)	0.43860410751211154
  (1, 14)	0.43860410751211154
  (1, 21)	0.2750363685138143
  (1, 50)	0.2520855255053467
  (2, 32)	0.4208122865191767
  (2, 33)	0.6107919421788647
  (2, 18)	0.6107919421788647
  (2, 6)	0.2770982249189687
  (3, 22)	0.35014528605914114
  (3, 52)	0.35014528605914114
  (3, 20)	0.35014528605914114
  (3, 54)	0.35014528605914114
  (3, 51)	0.535518256916423
  (3, 53)	0.301952473284094
  (3, 6)	0.15885055209572455
  (3, 5)	0.2412367096966542
  (3, 21)	0.21956631568316437
  (4, 42)	0.335436591890706
  (4, 41)	0.335436591890706
  :	:
  (10, 0)	0.3628432626653525
  (10, 48)	0.277469380986608
  (10, 32)	0.24998512990463073
  (10, 6)	0.1646112482258863
  (10, 50)	0.2085423573981102
  (11, 10)	0.37565386534262213
  (11, 27)	0.37565386534262213
  (11, 8)	0.37565386534262213
  (11, 29)	0.37565386534262213


## Originality Algo 3
### tf-idf scikit-learn + clustering
### participant as document

In [120]:
def tfidf_scikit_learn_clustering(cleaned_responses):
    tfIdfVectorizer=TfidfVectorizer(use_idf=True)
    tfIdf = tfIdfVectorizer.fit_transform(cleaned_responses)
#     df = pd.DataFrame(tfIdf.toarray(), columns=tfIdfVectorizer.get_feature_names())
#     df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
#     df = df.sort_values('TF-IDF', ascending=False)
#     print (df)
    
    number_of_clusters=2
    kmeans = KMeans(n_clusters=2).fit(tfIdf)
    
    order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
    terms = tfIdfVectorizer.get_feature_names()
    for i in range(number_of_clusters):
        top_ten_words = [terms[ind] for ind in order_centroids[i, :5]]
        print("Cluster {}: {}".format(i, ' '.join(top_ten_words)))


In [124]:
def get_originality_tfidf_scikit_learn_clustering(df, stopwords_list, remove_sw, join_list):
    originality_rating_df = get_cleaned_responses(df, stopwords_list, remove_sw, join_list)
    responses = []
    
    id_list = get_id_list(df)
    
    for participant in id_list:
        temp_df = originality_rating_df.loc[originality_rating_df['id'] == participant]
        temp_list = temp_df['response_processed'].tolist()
        temp_list = ' '.join(temp_list)
        responses.append(temp_list)
    
    tfidf_scikit_learn_clustering(responses)   

In [125]:
get_originality_tfidf_scikit_learn_clustering(data_brick, stopwords_spacy, True, True)

Cluster 0: weight weapon house door break
Cluster 1: use paperweight build throw demolish


### Write Results into Excel Sheet

In [18]:
prompts_list = ['cup', 'key', 'rope', 'brick', 'chair', 'pencil', 'shoe']
data_list = [data_cup, data_key, data_rope, data_brick, data_chair, data_pencil, data_shoe]

In [19]:
# write results df of each dataset for specific methods
def write_results_excel(method, stopwords):
    # change this when writing new sheet
    writer = pd.ExcelWriter('originality_tf_only_results.xlsx', engine='xlsxwriter')

    for i in range(len(prompts_list)):
        df = method(data_list[i], stopwords)
        df.to_excel(writer, sheet_name = prompts_list[i], index = False)
    writer.save()

In [20]:
# write_results_excel(get_originality_tf_only, stopwords_spacy)

In [21]:
# write_results_excel(get_originality_tfidf_scikit_learn, stopwords_spacy)

In [ ]:
# write_results_excel(get_originality_tfidf_scikit_learn_clustering, stopwords_spacy)

Algo Design Brainstorming:
* Algo Design w/o scikit-learn
    * asdlfjakl;fjka
    * asdfasf

* Algo Design w/ scikit-learn

* To Do List

